In [80]:
from pathlib import Path
from ebooklib import epub

In [81]:
def resort(paths):
    resort_ = dict(zip("9173258640","0123456789"))
    new_paths = [paths[int(resort_[str(idx)])] for idx, item in enumerate(paths)]
    return new_paths

print(Path('./sxft_data').iterdir())
print([i for i in Path('./sxft_data').iterdir()])
print(resort([i for i in Path('./sxft_data').iterdir()])[0].stem)
print(list(Path('./sxft_data/第一卷').glob('**/*.txt')))

[WindowsPath('sxft_data/真结局'), WindowsPath('sxft_data/第一卷'), WindowsPath('sxft_data/第七卷'), WindowsPath('sxft_data/第三卷'), WindowsPath('sxft_data/第二卷'), WindowsPath('sxft_data/第五卷'), WindowsPath('sxft_data/第八卷'), WindowsPath('sxft_data/第六卷'), WindowsPath('sxft_data/第四卷'), WindowsPath('sxft_data/第零卷')]
第零卷
[WindowsPath('sxft_data/第一卷/本卷简介.txt'), WindowsPath('sxft_data/第一卷/第001章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第002章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第003章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第004章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第005章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第006章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第007章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第008章/本章内容.txt'), WindowsPath('sxft_data/第一卷/第009章/本章内容.txt')]


In [ ]:
def create_epub(root_folder, book_name='兽血沸腾'):
    # 创建一个新的EPUB书籍对象
    book = epub.EpubBook()
    
    # 设置元数据
    book.set_identifier('id-mos-2025-1120-0247')
    book.set_title(f"{book_name} 电子书")
    book.set_language('zh-CN')
    book.add_author('未知作者')

    # 初始化章节列表
    chapters = []

    # 遍历第一级目录下的每个卷
    for volume in resort([i for i in Path(root_folder).iterdir()]):
        
        # 遍历第二级目录下的每个章节
        for chapter_file in volume.iterdir():

            if chapter_file.is_file():
                volume_name_front = volume.stem

                # 读取章节内容
                with open(chapter_file, 'r', encoding='ansi') as f:
                    content = f.read().split('\n')
                    volume_name = volume_name_front + ' ' + content[0]
                    # content[0] = volume_name
                    content = '<p>' + '</p><p>'.join(content[1:]) + '</p>'

                # 创建章节对象
                chapter = epub.EpubHtml(title=volume_name,
                                    file_name=f'{volume.stem}.xhtml',
                                    lang='zh-CN')
                chapter.content = f'<h1>{volume_name}</h1>{content}'

                # 添加章节到书籍
                book.add_item(chapter)
                chapters.append(chapter)

            if chapter_file.is_dir():
                Chapter_file = chapter_file
                chapter_file = list(chapter_file.glob('**/*.txt'))[0]
                
                # 读取章节内容
                with open(chapter_file, 'r', encoding='ansi') as f:
                    content = f.read().split('\n')
                    chapter_name = Chapter_file.stem + ' ' + content[0]
                    # content[0] = chapter_name
                    content = '<p>' + '</p><p>'.join(content[1:]) + '</p>'

                # 创建章节对象
                chapter = epub.EpubHtml(title=chapter_name,
                                    file_name=f'{volume.name}_{Chapter_file.stem}.xhtml',
                                    lang='zh-CN')
                chapter.content = f'<h1>{volume_name} {chapter_name}</h1>{content}'
                
                # 添加章节到书籍
                book.add_item(chapter)
                chapters.append(chapter)

    # 定义Table of Contents (TOC)
    book.toc = tuple(chapters)

    # 添加默认NCX and Nav files
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # 定义CSS样式
    style = '''
@namespace html url(http://www.w3.org/1999/xhtml);
body { font-family:Times, Times New Roman, serif; }
'''
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

    # 将CSS添加到书籍
    book.add_item(nav_css)

    # 定义SPINE
    book.spine = ['nav'] + chapters

    # 写入EPUB文件
    epub.write_epub(f'{book_name}.epub', book, {})


In [120]:
if __name__ == "__main__":
    root_folder = Path('./sxft_data')  # 替换为你的根文件夹路径
    create_epub(root_folder)